<a href="https://colab.research.google.com/github/YKannikanti/Yasasri-/blob/main/midterm/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [14]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [15]:
def calculate_bmi(height_m, weight_kg):
    """
    calculate Body Mass Index (BMI)

    BMI = Weight (kg)/ (height (m)^2)

    >>> calculate_bmi(70,1.75)
    22.86
    >>> calculate_bmi(90,1.6)
    35.16
    >>> calculate_bmi(0,1.75)
    Traceback (most recent call list):
       ...
    ValueError: Weight and Height cannot be zero
    >>> calculate_bmi(1.75,0)
    Traceback (most recent call list):
       ...
    ValueError: Weight and Height must be greater than zero
    """
    if height_m == 0 or weight_kg == 0:
        raise ValueError("Weight and Height must be greater than zero")
    bmi=weight_kg / (height_m ** 2)
    return round(bmi,2)


### Step 2: Calculate BODE Score

In [16]:
def calculate_bode_score(fev_pct, bmi, distance_in_meters,dyspnea_description):
    """
    calculate BODE Score based on the BMI,FEV1%,dyspnea,and 6-minute walk distance.

    >>> calculate _bode_score(22,40,'SEVERE',300)
    4
    >>> calculate_bode_score(30,60,MILD,200)
    1
    """
    bode_score=0

    # BMI POINTS
    if bmi > 21:
        bode_score += 0
    else:
        bode_score += 1


    # FEV1% points
    if fev_pct >=65:
        bode_score += 0
    elif 50 <= fev_pct < 65:
        bode_score += 1
    elif 36 <= fev_pct < 50:
        bode_score += 2
    else:
        bode_score += 3

    # Dyspnea points
    if dyspnea_description == 'NONE':
        bode_score += 0
    elif dyspnea_description == 'MILD':
        bode_score += 1
    elif dyspnea_description == 'MODERATE':
        bode_score += 2
    else:
        bode_score += 3

    # Distance walked points
    if distance_in_meters < 350:
        bode_score += 0
    elif 250 <= distance_in_meters < 350:
        bode_score += 1
    elif 200 <= distance_in_meters < 250:
        bode_score += 2
    else:
        bode_score += 3

    return bode_score


### Step 3: Calculate BODE Risk

In [17]:
def calculate_bode_risk(bode_score):
    """
    calculate BODE Survival Risk based on the BODE Score.

    >>> calculate_bode_risk(1)
    '80-90%'
    >>> calculate_bode_risk(4)
    '35-50%'
    """
    if bode_score == 0 or bode_score == 1:
        return '80-90%'
    elif bode_score == 2 or bode_score == 3:
        return '70-80%'
    elif bode_score == 4 or bode_score == 5:
        return '50-70%'
    elif bode_score == 6 or bode_score == 7:
        return '35-50%'
    else:
        return 'HIGH RISK,< 35%'

### Step 4: Load Hospital Data

In [18]:
def load_patient_data(file_path):
    """
    Load patient data from a CSV file and return a list of dictionaries.
    Each dictionary represents a patient.

    Returns:
    A list of patient data dictionaries.
    """
    patients= []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            patients.append(row)
    return patients

### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [19]:
def process_patients(patients_csv,patient_output_file,hospital_json,hospital_output_file):
    # Load patient data
    patients = load_patient_data(patients_csv)

    patient_results = []
    hospital_data = {}

    # Process each patient
    for patient in patients:
        #convert inputs
        height_m = float(patient['HEIGHT_M'])
        weight_kg = float(patient['WEIGHT_KG'])
        fev_pct = float(patient['Fev_pct'])
        dyspnea_description = patient['dyspnea_description']
        distance_in_meters = float(patient['distance_in_meters'])
        hospital = patient['hospital']

        # Calculate BMI
        bmi = calculate_bmi(height_m, weight_kg)

        # Calculate BODE score
        bode_score = calculate_bode_score(fev_pct, bmi, distance_in_meters,dyspnea_description)

        # Calculate BODE risk
        bode_risk = calculate_bode_risk(bode_score)

        # store the patient result
        patient_results.append({
            'NAME': patient['NAME'],
            'BODE_SCORE': bode_score,
            'BODE_RISK': bode_risk,
            'HOSPITAL': hospital
        })

        # store hospital data
        if hospital not in hospital_data:
            hospital_data[hospital] = {
                'copd_count': 0,
                'total_score': 0,
                'total_risk': []
        }


        hospital_data[hospital]['copd_count'] += 1
        hospital_data[hospital]['total_score'] += bode_score
        hospital_data[hospital]['total_risk'].append(bode_risk)

        # Save patient output
        with open(patient_output_file, 'w', newline='') as  file:
            writer = csv.Dictwriter(file,fieldnames=['NAME','BODE_SCORE','BODE_RISK','HOSPITAL'])
            writer.writeheader()
            writer.writerows(patient_results)

        # Process hospital statistics
        hospital_results = []
        for hospital, data in hospital_data.items():
            avg_bode_score = data['total_bode_score'] / data['copd_count']
            avg_risk ='/'.join(set(data['total_risk'])) #simplified for now


            hospital_results.append({
                'HOSPITAL_NAME': hospital,
                'COPD_COUNT': data['copd_count'],
                'PCT_OF_COPD_CASES_OVER_BEDS': None, # This requires bed data which is n't provided.
                'AVG_SCORE': round(avg_bode_score,2),
                'AVG_RISK': avg_risk
            })

        # save hospital output
        with open(hospital_output_file, 'w', newline='') as file:
            writer = csv.Dictwriter(file, fieldnames=['HOSPITAL_NAME', 'COPD_COUNT', 'PCT_OF_COPD_CASES_OVER_BEDS', 'AVG_SCORE', 'AVG_RISK'])
            writer.writeheader()
            writer.writerows(hospital_results)